In [1]:
import os

In [2]:
os.chdir("made-ml-demo-app-model")

In [3]:
import json
import pathlib
import pickle
import random

from IPython.display import display
import ipywidgets as widgets 
import torch
import trimesh
import numpy as np
from trimesh import creation
from torch_geometric.data  import Batch
from torch_geometric import utils
from pytransform3d.transformations import random_transform
from matplotlib import pyplot as plt
import matplotlib


from point_cloud_cls import SimpleClsLDGCN, BaseTransform
from train_param import DataParams

%matplotlib inline


matplotlib.rcParams["figure.figsize"] = (10, 10)
matplotlib.rcParams["font.size"] = 16

In [4]:
label_encoder_dump_filepath = pathlib.Path("inference-data", "label_encoder", "label_encoder.pickle")

In [5]:
with open(label_encoder_dump_filepath, "rb") as dump_file:
    label_encoder = pickle.load(dump_file)

In [6]:
path_to_model_config = pathlib.Path("inference-data", "model", "model_params.json")

In [7]:
with open(path_to_model_config, "r") as _model_params_file:
    model_params = json.load(_model_params_file)

In [8]:
cls_model = SimpleClsLDGCN(**model_params)

In [9]:
del model_params

In [10]:
path_to_checkpoint = pathlib.Path("inference-data", "model", "model_state.pth")

In [11]:
device = "cpu"

In [12]:
inference_transform = BaseTransform(DataParams().num_points)

In [13]:
cls_model.load_state_dict(torch.load(path_to_checkpoint, map_location=device))
cls_model.eval();

# Демострация работы приложения для классификации 3D примитивов 

На основе [PyTorchGeometric](https://github.com/rusty1s/pytorch_geometric) была обучена модель для классификации 3D примитивов.

В обучениие использовались следующие классы:

In [14]:
print(*label_encoder.classes_, sep="\n")

cone
cube
cylinder
plane
torus
uv_sphere


Все примитивы представлены в виде полигональных моделей. Прежде чем классифицировать примитив он преобразуется в "облако точек". Используется следующее число точек, которое случайно выбираются на поверхности примитивов:

In [15]:
DataParams().num_points

512

In [16]:
def cone_generator(transform):
    radius = random.uniform(0.2, 2)
    height = random.uniform(0.2, 4)
    return creation.cone(radius, height, transform=transform)

def cube_generator(transform):
    size = random.uniform(0.1, 2)
    return creation.box((size, size, size), transform=transform)

def cylinder_generator(transform):
    radius = random.uniform(0.2, 2)
    height = random.uniform(1, 3)
    return creation.cylinder(radius, height, transform=transform)

def plane_generator(transform):
    vertices = np.array(
        (
            (1, 1, 0), (-1, 1, 0), (-1, -1, 0), (1, -1, 0)
        )
    )
    faces = np.array(
        (
            (0, 1, 2),
            (2, 3, 0),
            (2, 1, 0),
            (0, 3, 2)
        )
    )
    plane = trimesh.Trimesh(vertices=vertices, faces=faces)
    plane.apply_transform(transform)
    return plane

def sphere_generator(transform):
    radius = random.uniform(1, 10)
    return creation.icosphere(radius=radius, subdivisions=2)

def torus_generator(transform):
    torus_radius = random.uniform(0.1, 5)
    dist = random.uniform(torus_radius + 0.1,  torus_radius + 10)
    circle = trimesh.path.creation.circle(torus_radius, (dist, 0))
    return creation.revolve(circle.discrete[0], transform=transform)

In [17]:
translate_mapping = {
    "cone": "конус",
    "cube": "куб",
    "cylinder": "цилиндр",
    "plane": "плоскость",
    "uv_sphere": "сфера",
    "torus": "тор"
}

generators = {
    "конус": cone_generator,
    "куб": cube_generator,
    "цилиндр": cylinder_generator,
    "плоскость": plane_generator,
    "сфера": sphere_generator,
    "тор": torus_generator
}

In [18]:
labels = tuple(generators.keys())

label_widget = widgets.Label("Выбор 3D примитива для классификации:")

model_list_widget = widgets.ToggleButtons(
    options=labels,
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

del labels

In [19]:
# material template in Viola does not render dropdown list. ToggleButtons choosed instead
primitive_choosing = widgets.GridBox([model_list_widget])

In [20]:
header_widget = widgets.VBox([label_widget, primitive_choosing])

In [21]:
score = None
output_plot_widget = widgets.Output()
output_plot_widget.layout.width = "40%"
output_mesh_widget = widgets.Output()
output_mesh_widget.layout.border = "solid"
output_mesh_widget.layout.width = "60%"
output_widget = widgets.HBox([output_plot_widget, output_mesh_widget])
output_widget.layout.height = "600px"

In [22]:
RED_COLOR = (255, 0, 0)
GREEN_COLOR = (0, 255, 0)

In [23]:
@torch.no_grad()
def classify_object(mesh, model, transform):
    data = utils.from_trimesh(mesh)
    batch = Batch.from_data_list([transform(data)])
    prediction_score = model.forward(batch)
    return prediction_score

In [24]:
def generate_and_calassify_primitive(label,
                       generator,
                       model,
                       inference_transform,
                       label_encoder,
                       translate_mapping: dict):
    random_transformation = random_transform()
    # only rotate
    random_transformation[:-1, -1] = 0
    mesh = generator[label](random_transformation)
    raw_score = classify_object(mesh, model, inference_transform)[0]
    pred_label = label_encoder.inverse_transform([raw_score.argmax()])[0]
    
    if label == translate_mapping[pred_label]:
        color = GREEN_COLOR
    else:
        color = RED_COLOR

    mesh.visual = trimesh.visual.ColorVisuals(mesh=mesh, vertex_colors=color)
    global score
    score = torch.exp(raw_score)
    with output_mesh_widget:
        display(mesh.show(viewer="notebook", smooth=True))

In [25]:
def plot_score(label_encoder, translate_mapping):
    x = tuple(range(len(score)))
    with output_plot_widget:
        plt.bar(x, score, tick_label=tuple(map(lambda x: translate_mapping[x], label_encoder.classes_)))
        plt.title("Уверенность модели")
        plt.grid(True)
        plt.show()

In [26]:
classify_button = widgets.Button(
    description="Классифицировать примитив",
    disabled=False,
    tooltip="Классифицировать выбранный примитив",
    button_style=""
)

classify_button.layout.width = "auto"

In [27]:
def calassify_callback(button):
    output_plot_widget.clear_output()
    output_mesh_widget.clear_output()
    generate_and_calassify_primitive(model_list_widget.value
                                     , generator=generators
                                     , model=cls_model
                                     , inference_transform=inference_transform
                                     , label_encoder=label_encoder
                                     , translate_mapping=translate_mapping)
    plot_score(label_encoder, translate_mapping)

In [28]:
classify_button.on_click(calassify_callback)

# Классификация случайно выбранного примитива 

In [29]:
widgets.VBox([header_widget, classify_button, output_widget])